In [ ]:
#Generate dataset

!python -m generate_final_dataset --out data_dataset_final/collector_health_data.csv --minutes 100000 --seed 42

In [ ]:
#this code runs cross validation on complex dataset with new target feature estimation (gpt, better)

!python -m cross_fold_eval_new \
  --data data_dataset_final/collector_health_data.csv \
  --folds 5 \
  --epochs 25 \
  --batch 512 \
  --ema_window 5 \
  --timesteps 15 \
  --label_mode horizon_binary \
  --horizon_min 120 \
  --outdir out_evaluate_cross_val_new_comp_100000

In [ ]:
!python -m train_proactive_model \
    --data data_dataset_final/collector_health_data.csv \
    --outdir out_risk_model_new \
    --horizon 120

In [ ]:
!python -m explain_alert \
    --data data_dataset_final/collector_health_data.csv \
    --model_path out_risk_model_new/risk_model_proactive_h120.h5 \
    --alert_index 95000 \
    --outdir shap_explanation

#The scripts use an 80/20 split for train/test. So, if the full dataset has 100,000 rows, it would train on the first 80,000, 
#and any index for explanation from the remaining 20,000 (e.g., index 95,000) would be a valid test data point.

In [ ]:
!python -m train_multi_horizon \
    --data data_dataset_final/collector_health_data.csv \
    --outdir out_multi_horizon_models \
    --horizons 30 60 120

In [ ]:
!python -m evaluate_multi_horizon \
    --data data_dataset_final/collector_health_data.csv \
    --models_dir out_multi_horizon_models \
    --event_index 0 \
    --outdir out_multi_horizon_eval